In [1]:
# !pip install paramiko
# !pip install scp

In [21]:
import ast
import json
import os
import boto3
import subprocess
import numpy as np
import time
import datetime
import paramiko
import io
from scp import SCPClient, SCPException
import sys
from ast import literal_eval

In [2]:
np.set_printoptions(threshold=sys.maxsize)

In [3]:
INSTANCE_SIZE = 5

In [4]:
client = boto3.client('ec2', region_name='us-east-1')
# Create SQS client
sqs = boto3.resource('sqs')

In [66]:
def get_default_security_group(client, key_name):
    #extract key_name attribute from the security groups returned
    response = [group[key_name] for group in client.describe_security_groups()['SecurityGroups'] if group['GroupName'] == 'default']

    return response

def get_key_pairs(client, removeExisting=False):
    if removeExisting:
        client.delete_key_pair(KeyName='airscholar-key')

    keypairs = client.describe_key_pairs()['KeyPairs']
    keypair = list(filter(lambda x: x['KeyName'] == 'airscholar-key', keypairs))

    if not keypair:
        keypair = client.create_key_pair(KeyName='airscholar-key')
        f = io.StringIO(keypair['KeyMaterial'])
        data = f.read()
        file = open('labsuser.pem', 'w')
        file.write(data)
        file.close()
    else:
        keypair = keypair[0]

    return keypair

def launch_new_instance(client, keypair, count):
    response = client.run_instances(
        ImageId='ami-05723c3b9cf4bf4ff',
        InstanceType='t2.micro',
        KeyName=keypair,
        MaxCount=count,
        MinCount=count,
        Monitoring={
            'Enabled': True
        },
        SecurityGroupIds= get_default_security_group(client, key_name='GroupId')
    )
    ec2_inst_ids = [res["InstanceId"] for res in response]
    waiter = client.get_waiter('instance_running')
    waiter.wait(InstanceIds=[ec2_inst_ids])
    return ec2_inst_ids

def prepare_instances(client, keypair, count):
    ec2 = boto3.resource('ec2')
    ec2_inst_ids = []
    # print(ec2.instances.all())

    for instance in ec2.instances.all():
        if instance.state['Name'] == 'running':
            ec2_inst_ids.append(instance.id)

    if not ec2_inst_ids:
        ec2_inst_ids.append(launch_new_instance(client, keypair, count))

    return ec2, ec2_inst_ids

def configure_ssh():
    sshs = []
    for count in range(0, INSTANCE_SIZE):
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        sshs.append(ssh)
    return sshs

def ssh_connect_with_retry(ssh, ip_address, retries):
    if retries > 3:
        return False
    f = open('labsuser.pem', 'r')
    privkey = paramiko.RSAKey.from_private_key(f)
    # print(privkey)
    interval = 5
    try:
        retries += 1
        print('SSH into the instance: {}'.format(ip_address))
        ssh.connect(hostname=ip_address,
                    username='ec2-user', pkey=privkey)
        return True
    except Exception as e:
        print(e)
        time.sleep(interval)
        print('Retrying SSH connection to {}'.format(ip_address))
        ssh_connect_with_retry(ssh, ip_address, retries)

def ssh_disconnect(ssh):
        """Close ssh connection."""
        if ssh:
            ssh.close()

def get_public_address(ec2, instance_id):
    # ec2 = boto3.resource('ec2', region_name='us-east-1')
    instance = ec2.Instance(id=instance_id)
    instance.wait_until_running()
    current_instance = list(ec2.instances.filter(InstanceIds=[instance_id]))
    ip_address = current_instance[0].public_ip_address
    return ip_address

def get_queue(sqs, queue_name):
    # Get the queue. This returns an SQS.Queue instance
    # There is no queue, create a new SQS queue
    attributes = {
        'DelaySeconds': '0',
        'MessageRetentionPeriod': '86400',
        "ReceiveMessageWaitTimeSeconds": "0"
    }

    for idx in range(INSTANCE_SIZE):
        sqs.create_queue(
            QueueName=f"{queue_name}{idx}",
            Attributes=attributes
        )
    sqs.create_queue(
        QueueName='result-queue',
        Attributes=attributes
    )

def send_message_to_queue(sqs, queue_name, message):
    queue = sqs.get_queue_by_name(QueueName=queue_name)

    # Send message to SQS queue
    response = queue.send_messages(
        Entries=message
    )
    return response

def install_required_packages(ssh):
    stdin, stdout, stderr = ssh.exec_command("sudo yum install pip -y && pip install numpy")
    astdin, stdout, stderr = ssh.exec_command("sudo yum install pip -y && pip install numpy")
    return stdout, stderr

def get_messages_from_queue(queue_url):
    sqs_client = boto3.client('sqs')

    messages = []

    while True:
        resp = sqs_client.receive_message(
            QueueUrl=queue_url
        )

        try:
            messages.extend(resp['Messages'])
        except KeyError:
            break

        entries = [
            {'Index': msg['MatrixId'], 'Matrix': msg['Matrix']}
            for msg in resp['Messages']
        ]

        resp = sqs_client.delete_message_batch(
            QueueUrl=queue_url, Entries=entries
        )

        if len(resp['Successful']) != len(entries):
            raise RuntimeError(
                f"Failed to delete messages: entries={entries!r} resp={resp!r}"
            )

    return messages

def split_row(array, nrows, ncols):
    """Split a matrix into sub-matrices."""

    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))

def split_col(array, nrows, ncols):
    """Split a matrix into sub-matrices."""
    r, h = array.shape
    return [np.vsplit(i, 5) for i in np.hsplit(arr1, r)]

def generate_array(nrows, ncols):
    arr = np.random.randint(10, size=(nrows, ncols))
    # # print('arr 1:\n', arr)
    # arr = split_row(arr, 1, split_size)
    arr1 = np.random.randint(10, size=(nrows, ncols))
    # print('arr 2:\n', arr1)
    # arr1 = split_col(arr1, 1, split_size)

    return arr, arr1

def upload_file_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def bulk_upload(scp, filepaths: list[str], remote_path, host):
        """
        Upload multiple files to a remote directory.

        :param List[str] filepaths: List of local files to be uploaded.
        """
        try:
            scp.put(
                filepaths,
                remote_path=remote_path,
                recursive=True
            )
            print(f"Finished uploading {len(filepaths)} files to {remote_path} on {host}")
        except SCPException as e:
            print(f"SCPException during bulk upload: {e}")
        except Exception as e:
            print(f"Unexpected exception during bulk upload: {e}")

def configure_aws_access_for_ssh(ssh, ip_address):
    """
    This function extracts the AWS configuration you have locally and push to the server
    :param ssh:ssh object
    :return:
    """
    output = subprocess.getoutput("cat ~/.aws/credentials")
    ssh.exec_command(f'mkdir ~/.aws && touch ~/.aws/credentials')
    ssh.exec_command(f"echo '{output}' > ~/.aws/credentials")
    print(f'SSH AWS configuration done for {ip_address}')

def matrix_dot_product(matrix_a, matrix_b):
    start_time = datetime.datetime.now()
    result = []
    for i in range(len(matrix_a)):
        row = []
        for j in range(len(matrix_b[0])):
            sum = 0
            for k in range(len(matrix_b)):
                sum += matrix_a[i][k] * matrix_b[k][j]
            row.append(sum)
        result.append(row)
    print('Computation time', datetime.datetime.now() - start_time)

    return result

def matrix_add(matrix_1, matrix_2):
    start_time = datetime.datetime.now()
    result = []
    for idx_row in range(0, len(matrix_1)):
        row = matrix_1[idx_row]
        row1 = matrix_2[idx_row]
        cols = []
        for idx_col in range(0, len(row)):
            cols.append(row[idx_col] + row1[idx_col])
        result.append(cols)
    print('Computation time', datetime.datetime.now() - start_time)
    return result

In [6]:
QUEUE_NAME = 'queue'

In [7]:
get_queue(sqs, QUEUE_NAME)

In [8]:
sshs = configure_ssh() 
keypair = get_key_pairs(client, False)
ec2, instances = prepare_instances(client, keypair['KeyName'], 5)
ip_addresses = [get_public_address(ec2, instance) for instance in instances]

for idx in range(0, len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    ssh_connect_with_retry(ssh, ip_address, 0)

SSH into the instance: 44.206.246.195
SSH into the instance: 54.86.255.244
SSH into the instance: 3.89.115.67
SSH into the instance: 3.82.172.51
SSH into the instance: 34.203.252.117


In [67]:
# ssh_disconnect(ssh)
for idx in range(len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    install_required_packages(ssh)
    configure_aws_access_for_ssh(ssh, ip_address)

SSH AWS configuration done for 44.206.246.195
SSH AWS configuration done for 54.86.255.244
SSH AWS configuration done for 3.89.115.67
SSH AWS configuration done for 3.82.172.51
SSH AWS configuration done for 34.203.252.117


In [11]:
# sqs = boto3.resource('sqs')
# sqs
# response = send_message_to_queue(sqs, QUEUE_NAME, "Hi there! This is my queue message4!", 1, 0)
# response = send_message_to_queue(sqs, QUEUE_NAME, 'X', 1, 0)

In [48]:
ARRAY_SIZE = 500

In [49]:
arr, arr1 = generate_array(ARRAY_SIZE,ARRAY_SIZE)

In [50]:
res = matrix_dot_product(arr, arr1)

In [131]:
INT_ARRAY_SIZE = int(ARRAY_SIZE/INSTANCE_SIZE)
s_arr = split_row(arr, INT_ARRAY_SIZE, INT_ARRAY_SIZE)

In [132]:
for row in range(0, INT_ARRAY_SIZE):
    print(s_arr[row])
    break

[[1 0 4 1 3 0 7 5 3 9 0 2 7 8 1 4 5 4 0 1]
 [1 1 6 7 4 8 7 1 4 9 2 1 0 5 9 7 5 1 2 9]
 [8 4 2 8 2 4 6 1 0 8 4 0 8 6 5 1 0 2 4 4]
 [2 8 1 6 7 0 4 2 5 2 9 8 9 4 3 6 3 3 3 0]
 [0 2 7 8 3 3 8 6 2 0 0 7 1 2 4 7 6 0 9 0]
 [3 7 9 1 1 0 3 7 1 6 8 6 0 3 8 3 3 8 2 2]
 [9 4 7 8 5 8 2 9 1 9 4 7 6 3 1 8 2 6 5 6]
 [4 5 2 0 7 6 7 4 4 1 1 5 7 8 6 1 1 4 1 1]
 [6 3 3 8 5 1 5 6 8 9 6 4 5 0 3 5 4 1 3 0]
 [3 7 1 9 1 3 4 3 1 0 6 9 7 9 4 2 9 1 7 2]
 [1 5 0 8 0 6 6 4 6 4 4 8 5 4 9 1 9 3 7 6]
 [6 2 0 0 0 0 5 2 1 8 3 6 8 7 3 0 2 6 6 4]
 [9 2 4 4 4 9 3 8 7 0 8 7 8 8 0 1 6 0 1 7]
 [9 5 8 4 9 3 1 9 8 6 8 8 4 2 3 2 2 2 9 7]
 [0 7 9 2 2 7 7 0 1 2 4 7 9 7 5 7 7 4 5 9]
 [4 9 8 7 2 2 2 4 4 7 4 6 3 7 3 5 9 0 7 7]
 [9 8 8 2 5 0 1 3 3 3 0 6 6 8 8 1 0 7 8 6]
 [9 2 9 0 8 1 2 7 7 2 4 2 3 4 9 4 9 4 0 5]
 [4 4 0 6 2 1 0 1 4 6 3 8 1 9 6 5 1 5 7 9]
 [5 9 3 2 8 3 5 0 8 9 3 3 4 7 0 1 6 3 6 0]]


In [ ]:
a[1][0]

In [134]:
# s_arr1 = split_col(arr1, 2, 5)
s_arr1 = split_row(arr1, INT_ARRAY_SIZE, INT_ARRAY_SIZE)

In [18]:
# iter = len(s_arr)/100

In [55]:

# b = str(b).replace('\n', ',').replace(' ', '')

In [57]:
# for y in range(0, 20)
# for x in range(0, iter):
# a = arr[0]
# b = arr[1]
# a = str(a).replace(' ', ',').replace('\n', '')
# b = str(b).replace('\n', ',').replace(' ', '')
    
    # stdin, stdout, stderr = ssh.exec_command(f'python dot_prod.py {a} {b}')
    # result = stdout.read().decode("utf-8").replace('\n', '')
    # literal_eval(result)[0][0]
    # print(stderr.read().decode("utf-8"))

In [46]:
scp = SCPClient(ssh.get_transport())

In [47]:
# fetch_local_files('.')

In [48]:
bulk_upload(scp, ['./dot_prod.py'], '~', ip_address)

Finished uploading 1 files to ~ on 54.152.58.160


In [148]:
def reformat_data(data):
    return str(data).replace(' ', ',').replace('\n', '')
# a = s_arr[0]
# b = s_arr[1]
# a = str(a).replace(' ', ',').replace('\n', '')
# b = str(b).replace(' ', ',').replace('\n', '')
messages = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(0, 5)]
messages1 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(5, 10)]
messages2 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(10, 15)]
messages3 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(15, 20)]
messages4 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(20, 25)]

In [149]:
messages

[{'Id': '0',
  'MessageBody': "('[[1,0,4,1,3,0,7,5,3,9,0,2,7,8,1,4,5,4,0,1],[1,1,6,7,4,8,7,1,4,9,2,1,0,5,9,7,5,1,2,9],[8,4,2,8,2,4,6,1,0,8,4,0,8,6,5,1,0,2,4,4],[2,8,1,6,7,0,4,2,5,2,9,8,9,4,3,6,3,3,3,0],[0,2,7,8,3,3,8,6,2,0,0,7,1,2,4,7,6,0,9,0],[3,7,9,1,1,0,3,7,1,6,8,6,0,3,8,3,3,8,2,2],[9,4,7,8,5,8,2,9,1,9,4,7,6,3,1,8,2,6,5,6],[4,5,2,0,7,6,7,4,4,1,1,5,7,8,6,1,1,4,1,1],[6,3,3,8,5,1,5,6,8,9,6,4,5,0,3,5,4,1,3,0],[3,7,1,9,1,3,4,3,1,0,6,9,7,9,4,2,9,1,7,2],[1,5,0,8,0,6,6,4,6,4,4,8,5,4,9,1,9,3,7,6],[6,2,0,0,0,0,5,2,1,8,3,6,8,7,3,0,2,6,6,4],[9,2,4,4,4,9,3,8,7,0,8,7,8,8,0,1,6,0,1,7],[9,5,8,4,9,3,1,9,8,6,8,8,4,2,3,2,2,2,9,7],[0,7,9,2,2,7,7,0,1,2,4,7,9,7,5,7,7,4,5,9],[4,9,8,7,2,2,2,4,4,7,4,6,3,7,3,5,9,0,7,7],[9,8,8,2,5,0,1,3,3,3,0,6,6,8,8,1,0,7,8,6],[9,2,9,0,8,1,2,7,7,2,4,2,3,4,9,4,9,4,0,5],[4,4,0,6,2,1,0,1,4,6,3,8,1,9,6,5,1,5,7,9],[5,9,3,2,8,3,5,0,8,9,3,3,4,7,0,1,6,3,6,0]]', '[[2,9,5,3,5,4,1,2,6,3,9,8,4,4,8,5,4,3,6,4],[7,9,6,0,9,2,4,8,3,2,4,2,8,2,8,7,9,4,0,6],[3,5,0,5,3,8,9,6,9,5,2,7,0,5,7,1,1,6,

In [150]:
for idx in range(0, 5):
    send_message_to_queue(sqs, f'queue{idx}', messages)

In [100]:
literal_eval('(array([[0, 9, 6, 1, 5]]), array([[2, 6, 4, 4, 9]]))')

ValueError: malformed node or string: <ast.Call object at 0x13e01fdf0>

In [73]:
get_messages_from_queue(queue_url)

KeyError: 'MatrixId'

In [204]:
# # sqs = boto3.resource('sqs')
# queue = sqs.(QueueName='airscholar-queue')
# for message in sqs.receive_message(
#             MaxNumberOfMessages=10):
#         # process message body
#         body = json.loads(message.body)
#         print(body)


In [143]:
# message = response['Messages']
# receipt_handle = message['ReceiptHandle']

# # Delete received message from queue
# sqs.delete_message(
#     QueueUrl=queue_url,
#     ReceiptHandle=receipt_handle
# )


In [196]:
# print(json.dumps(message, indent=4))

In [45]:
# # ec2_inst_id
# bucketName = 'airscholar-mlbd-bucket'

In [146]:
# s3 = boto3.client('s3')

In [ ]:
# response = s3.get_object(Bucket=bucketName,
#                          Key='data.json')
# print("Done, response body:")
# print(response['Body'].read())

In [ ]:
#create instance
#configure instance
#create matrix
#split matrix
#send matrix to the queue
#read matrix from the queue on the instance created
#compute matrix
#send result to base


In [59]:
!git add .
!git commit -am "Update infrastructure implementation"
!git push --set-upstream origin main

[main d2098e2] Update infrastructure implementation
 1 file changed, 37 insertions(+), 3 deletions(-)
Enumerating objects: 35, done.
Counting objects: 100% (35/35), done.
Delta compression using up to 8 threads
Compressing objects: 100% (28/28), done.
Writing objects: 100% (31/31), 12.27 KiB | 6.14 MiB/s, done.
Total 31 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 1 local object.
To https://github.com/airscholar/MLCloudComputing-python.git
   3576c57..d2098e2  main -> main
branch 'main' set up to track 'origin/main'.
